In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch
import warnings
import logging
from torch.utils.data import Dataset
import json
import csv
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from pathlib import Path
import os

In [2]:
! pip install transformers datasets

In [2]:
!pip install transformers

In [3]:
data = pd.read_csv('/kaggle/input/empathetic-dialogues-facebook-ai/emotion-emotion_69k.csv')
#data = data[:10000]

In [4]:
class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = csv.reader(open(path, "r"))
        self.X = []
        for situation, label in zip(data['Situation'], data['labels']):
          conc = "<startofstring> "+situation+" <bot>: "+label+" <endofstring>"
          self.X.append(conc)
        #self.X = self.X[:5000]
        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>","bos_token": "<startofstring>", "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

In [7]:
def train(chatData, model, optim):
    epochs = 10
    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
chatData = ChatData("/kaggle/input/empathetic-dialogues-facebook-ai/emotion-emotion_69k.csv", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)
model.train()
optim = Adam(model.parameters(), lr=1e-3)
print("training .... ")
train(chatData, model, optim)

training .... 


100%|██████████| 10/10 [1:50:39<00:00, 663.92s/it]


In [8]:
def infer(inp):
    logging.getLogger("transformers").setLevel(logging.ERROR)
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, pad_token_id=tokenizer.eos_token_id, attention_mask=a )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    user_input, response = output.split("<bot>:")
    user_input = user_input.strip()
    response = response.strip()
    return response

In [12]:
while True:
  inp = input("User: ")
  if inp == 'exit':
    break
  print("Chatbot: ", infer(inp))

User:  last few days i feel lonely but i don't know why


Chatbot:  I'm sorry to hear that


User:  i always feel that if my family stay with me


Chatbot:  That's great. It's good to


User:   tomorrow i have a cricket match and i love cricket more than anything else


...

User:  Losing a job can be a tremendous blow, both financially and emotionally


Chatbot:  Ouch, that


User:  Sometimes life throws us unexpected curveballs that can knock us off our feet


Chatbot:  I am not sure


User:  exit
